<a href="https://colab.research.google.com/github/atharvarya12/Computer-Vision-Projects/blob/main/Week1Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import reapy
from reapy import reascript_api as RPR

# list of functions that are here:-
  1. add marker position
  2. move to next marker
  3. move to previous marker
  4. delete marker(given the marker index)
  5. change the output bus
  6. set input monitor mode
  7. opening fade in fade out settings
  8. applying fade in/fade out to an item in a particular track
  9. added one more fuction to change default fade in fade out properties.
  9. split track at given start point and end point
  10. split track at the cursor location
  11. trim the track
  12. copy part of the track
  13. cut part of the track
  14. paste

# MIDI Fuctions are not done yet


# Marker Functions

In [ ]:
#This fuction will add marker to set loction given by the user and set a name to the marker

def add_marker_at_position(marker_name, position):
    RPR.SetEditCurPos2(0, position, False, False)

    project = reapy.Project()
    marker_index = project.add_marker(position, marker_name)

    if marker_index != -1:
        reapy.print("Marker '{}' added at position: {}".format(marker_name, position))
    else:
        reapy.print("Failed to add marker at position: {}".format(position))

# Prompt the user for the marker name and position
marker_name = input("Enter the marker name: ")
position = float(input("Enter the position in seconds: "))

# Call the function to set the cursor position and add a marker at the specified position
add_marker_at_position(marker_name, position)

In [ ]:
#this function will move the cursor to the next marker location

def move_to_next_marker():
    cursor_position = RPR.GetCursorPositionEx(0)
    project_id = RPR.EnumProjects(-1, None, 0)[0]
    num_markers = RPR.CountProjectMarkers(project_id, 0, 0)[0]

    next_marker_position = float('inf')

    for marker_index in range(num_markers):
        _, _, _, _, marker_position, _, _, _ = RPR.EnumProjectMarkers2(project_id, marker_index, False, 0.0, 0.0, "", -1)

        if marker_position > cursor_position and marker_position < next_marker_position:
            next_marker_position = marker_position

    if next_marker_position != float('inf'):
        RPR.SetEditCurPos2(0, next_marker_position, False, False)
        reapy.print("Moved to next marker.")
    else:
        reapy.print("No next marker found.")

# Call the function to move to the next marker
move_to_next_marker()


In [ ]:
#this function will move the cursor to the previous marker location

def move_to_previous_marker():
    cursor_position = RPR.GetCursorPositionEx(0)
    project_id = RPR.EnumProjects(-1, None, 0)[0]
    num_markers = RPR.CountProjectMarkers(project_id, 0, 0)[0]

    previous_marker_position = float('-inf')

    for marker_index in range(num_markers):
        _, _, _, _, marker_position, _, _, _ = RPR.EnumProjectMarkers2(project_id, marker_index, False, 0.0, 0.0, "", -1)

        if marker_position < cursor_position and marker_position > previous_marker_position:
            previous_marker_position = marker_position

    if previous_marker_position != float('-inf'):
        RPR.SetEditCurPos2(0, previous_marker_position, False, False)
        reapy.print("Moved to previous marker.")
    else:
        reapy.print("No previous marker found.")

# Call the function to move to the previous marker
move_to_previous_marker()


In [ ]:
#this function will delete the marker with specified index

def delete_marker(marker_index):
    project_id = RPR.EnumProjects(-1, None, 0)[0]
    num_markers = RPR.CountProjectMarkers(project_id, 0, 0)[0]

    if marker_index >= 0 and marker_index < num_markers:
        RPR.DeleteProjectMarkerByIndex(project_id, marker_index)
        reapy.print("Marker at index {} deleted.".format(marker_index))
    else:
        reapy.print("Invalid marker index.")

# Example usage
marker_index_to_delete = 1
delete_marker(marker_index_to_delete)

Total given function 4 - adding marker,  changing name of a marker, navigating btw marker, deleting a marker
Completed 3, Left 1 (changing name of the marker (as i'm unable to extract name of a marker))

code to get info of a marker -
import reapy.reascript_api as RPR

def enumerate_project_markers():
    project_id = RPR.EnumProjects(-1, None, 0)[0]
    num_markers = RPR.CountProjectMarkers(project_id, 0, 0)[0]
    print(num_markers)

    for marker_index in range(num_markers):
        retval, project_id, idx, isrgnOut, posOut, rgnendOut, nameOut, markrgnindexnumberOut = RPR.EnumProjectMarkers2(project_id, marker_index, False, 0.0, 0.0, "", -1)

        if retval:
            print("Marker Index: {}".format(idx))
            print("Is Region: {}".format(isrgnOut))
            print("Position: {}".format(posOut))
            print("Region End: {}".format(rgnendOut))
            print("Marker Name: {}".format(nameOut))
            print("Marker Region Index Number: {}".format(markrgnindexnumberOut))
            print()

 #Call the function to enumerate project markers

enumerate_project_markers()

# Input/Output Routing functions

In [ ]:
'''this function changes the output_bus of a given track (for master track the output_bus = 0 , 1 for first hardware output, and soo on)
INPUT PARAMETERS :- 1.Track Index
                    2.Output bus

'''

def set_output_destination(track_index, output_bus):
    project = reapy.Project()

    # Find the track by index
    track = project.tracks[track_index] if track_index < len(project.tracks) else None

    if track is None:
        reapy.print("Invalid track index.")
        return

    # Set the output destination of the track
    RPR.SetMediaTrackInfo_Value(track.id, "I_NCHAN", output_bus + 2)

    # Display a prompt in Reaper
    reapy.print("Output destination set for track {}.".format(track.name))

# Example usage

# Specify the track index
track_index = 0

# Specify the desired output bus (e.g., 0 for Master Track, 1 for first hardware output, etc.)
output_bus = 2

# Call the function to set the output destination for the track
set_output_destination(track_index, output_bus)


In [ ]:
#this function changes the monitoring state of a track default is set to 2 when monitoring is enabled while recording)


def set_track_input_monitor(track_index, monitor=2, ignore_group_flags=0):
    project = reapy.Project()

    # Check if the track index is valid
    if track_index < 0 or track_index >= len(project.tracks):
        reapy.print("Invalid track index.")
        return

    track = project.tracks[track_index]
    new_monitor_value = RPR.SetTrackUIInputMonitor(track.id, monitor, ignore_group_flags)
    return new_monitor_value

# Example usage

# Specify the track index
track_index = 1

# Set input monitoring to "Monitoring" (value: 1) and ignore grouping and selection ganging flags
monitor = 2
ignore_group_flags = 0

# Call the function to set the track's input monitoring
new_monitor_value = set_track_input_monitor(track_index, monitor, ignore_group_flags)

if new_monitor_value == -1:
    reapy.print("Error setting input monitoring.")
elif new_monitor_value == 0:
    reapy.print("Input monitoring disabled")
elif new_monitor_value == 1:
    reapy.print("Input monitoring enabled")
elif new_monitor_value == 2:
    reapy.print("Input monitoring enabled only while recording")
else:
    reapy.print("Unknown input monitoring value")



TOTAL Function - 3, Completed -2

Input mode is a bit confusing as the input changes for different users

# Audio editing

In [ ]:
from reapy import reascript_api as RPR
#this opens fade in and fade out setup window in reaper where the user can select what kind of fadeing effect they want.
# Open the REAPER preferences window
RPR.Main_OnCommand(40016, 0)

# Switch to the "Project" tab
RPR.Main_OnCommand(40013, 0)

# Expand the "Media Track Item" section
RPR.Main_OnCommand(40867, 0)

# Scroll to the "Media Track Item" section
RPR.Main_OnCommand(40182, 0)


In [ ]:
'''Too add fade in the item in a track or a track
 default fade in shape is linear
The fade-in shape input is an integer value, where:

0 represents a linear shape,
1 represents a slow start/end shape,
2 represents a fast start shape,
3 represents a fast end shape,
4 represents a Bezier shape.'''

def add_fade_in_to_media_item(track_index=0, item_index=0, fade_in_length=0.5, fade_in_shape = 0):
    # Get the project
    project = RPR.EnumProjects(-1, None, 0)[0]

    # Get the total number of tracks in the project
    num_tracks = RPR.CountTracks(project)

    # Check if the track index is valid
    if track_index < 0 or track_index >= num_tracks:
        print("Invalid track index.")
        return

    # Get the track at the specified index
    track = RPR.GetTrack(project, track_index)

    # Check if the item index is valid within the track
    num_items = RPR.CountTrackMediaItems(track)
    if item_index < 0 or item_index >= num_items:
        print("Invalid item index.")
        return

    # Get the media item at the specified index within the track
    media_item = RPR.GetTrackMediaItem(track, item_index)

    # Get the name of the media item
    media_item_name = RPR.GetSetMediaItemTakeInfo_String(RPR.GetTake(media_item, 0), "P_NAME", "", False)

    # Check if the media item name is not empty
    if media_item_name != "":
        # Get the start position of the media item
        start_position = RPR.GetMediaItemInfo_Value(media_item, "D_POSITION")

        # Calculate the fade-in position
        fade_in_position = start_position + fade_in_length

        # Set fade-in length and curve shape
        RPR.SetMediaItemInfo_Value(media_item, "D_FADEINLEN", fade_in_length)
        RPR.SetMediaItemInfo_Value(media_item, "C_FADEINSHAPE", fade_in_shape)

        # Set fade-in end position
        RPR.SetMediaItemInfo_Value(media_item, "D_FADEINEND", fade_in_position)

        print("Fade-in effect of length {} seconds and shape {} added to media item at index {} in track at index {}.".format(fade_in_length, fade_in_shape, item_index, track_index))
    else:
        print("Media item at index {} in track at index {} not found.".format(item_index, track_index))


# Prompt the user for track index, media item index, fade-in length, and fade-in shape
track_index = int(input("Enter the track index: "))
item_index = int(input("Enter the media item index: "))
fade_in_length = float(input("Enter the fade-in length in seconds: "))
fade_in_shape = int(input("Enter the fade-in shape (0: Linear, 1: Slow start/end, 2: Fast start, 3: Fast end, 4: Bezier): "))

# Call the function to add fade-in effect to the specific media item within the track
add_fade_in_to_media_item(track_index, item_index, fade_in_length, fade_in_shape)


In [ ]:
'''Too add fade out the item in a track or a track
 default fade out shape is linear
The fade-out shape input is an integer value, where:

0 represents a linear shape,
1 represents a slow start/end shape,
2 represents a fast start shape,
3 represents a fast end shape,
4 represents a Bezier shape.'''


def add_fade_out_to_media_item(track_index=0, item_index=0, fade_out_length=0.5, fade_out_shape=0):
    # Get the project
    project = RPR.EnumProjects(-1, None, 0)[0]

    # Get the total number of tracks in the project
    num_tracks = RPR.CountTracks(project)

    # Check if the track index is valid
    if track_index < 0 or track_index >= num_tracks:
        print("Invalid track index.")
        return

    # Get the track at the specified index
    track = RPR.GetTrack(project, track_index)

    # Check if the item index is valid within the track
    num_items = RPR.CountTrackMediaItems(track)
    if item_index < 0 or item_index >= num_items:
        print("Invalid item index.")
        return

    # Get the media item at the specified index within the track
    media_item = RPR.GetTrackMediaItem(track, item_index)

    if media_item is not None:
        take = RPR.GetTake(media_item, 0)

        # Get the end position of the media item
        item_end_position = RPR.GetMediaItemInfo_Value(media_item, "D_POSITION") + RPR.GetMediaItemInfo_Value(media_item, "D_LENGTH")

        # Calculate the fade-out position
        fade_out_position = item_end_position - fade_out_length

        # Add fade-out using automation items
        RPR.InsertAutomationItem(take, -1, fade_out_position, fade_out_length)

        # Set the fade-out shape
        RPR.SetMediaItemInfo_Value(media_item, "C_FADEOUTSHAPE", fade_out_shape)

        print("Fade-out effect of length {} seconds added to media item at index {} in track at index {} with shape {}.".format(fade_out_length, item_index, track_index, fade_out_shape))
    else:
        print("Media item at index {} in track at index {} not found.".format(item_index, track_index))


# Prompt the user for track index, media item index, fade-out length, and fade-out shape
track_index = int(input("Enter the track index: "))
item_index = int(input("Enter the media item index: "))
fade_out_length = float(input("Enter the fade-out length in seconds: "))
fade_out_shape = int(input("Enter the fade-out shape (0: Linear, 1: Slow start/end, 2: Fast start, 3: Fast end): "))

# Call the function to add fade-out effect to the media item
add_fade_out_to_media_item(track_index, item_index, fade_out_length, fade_out_shape)


In [ ]:
import reapy
from reapy import reascript_api as RPR

def split_items(track_index, start_time=0, end_time=1):
    # Create a new Reaper project object
    project = reapy.Project()

    # Get the total number of tracks in the project
    num_tracks = len(project.tracks)

    # Check if the track index is valid
    if track_index < 0 or track_index >= num_tracks:
        reapy.print("Invalid track index.")
        return

    # Get the track at the specified index
    track = project.tracks[track_index]

    # Get all items in the track
    items = track.items

    # Initialize a variable to track if any items were trimmed
    trimmed = False

    # Iterate over all items in the track
    for item in items:
        # Calculate the end time of the item
        item_end_time = item.position + item.length

        # Check if the item's start or end time falls within the specified range
        if item.position < end_time and item_end_time > start_time:
            # Calculate the new start and end positions for the item
            new_start = max(item.position, start_time)
            new_end = min(item_end_time, end_time)

            # Split the item at the new start position
            item.split(new_start - item.position)

            # Set the trimmed flag to True
            trimmed = True

    # Check if any items were trimmed and print the appropriate message
    if trimmed:
        reapy.print("Items splited on track at index: {}".format(track_index))
    else:
        reapy.print("No items found on track at index: {}".format(track_index))


# Example usage

# Get the track index
track_index = 0

# Set the start and end time (in seconds)
start_time = 1
end_time = 2

# Call the function to trim the specified part of the track
split_items(track_index, start_time, end_time)


In [ ]:


def split_items_at_cursor(track_index):
    # Create a new Reaper project object
    project = reapy.Project()

    # Get the current cursor position
    cursor_position = project.cursor_position

    # Print the cursor position
    reapy.print("Cursor position: {}".format(cursor_position))

    # Get the total number of tracks in the project
    num_tracks = len(project.tracks)

    # Check if the track index is valid
    if track_index < 0 or track_index >= num_tracks:
        reapy.print("Invalid track index.")
        return

    # Get the track at the specified index
    track = project.tracks[track_index]

    # Get all items in the track
    items = track.items

    # Initialize a variable to track if any items were split
    split = False

    # Iterate over all items in the track
    for item in items:
        # Calculate the end time of the item
        item_end_time = item.position + item.length

        # Check if the cursor position falls within the item's time range
        if item.position < cursor_position < item_end_time:
            # Split the item at the cursor position
            item.split(cursor_position - item.position)

            # Set the split flag to True
            split = True

    # Check if any items were split and print the appropriate message
    if split:
        reapy.print("Items split on track at index: {}".format(track_index))
    else:
        reapy.print("No items found on track at index: {}".format(track_index))


# Example usage

# Get the track index
track_index = 0

# Call the function to split the items at the cursor position
split_items_at_cursor(track_index)
split_items_at_cursor(track_index)

In [ ]:
#this function trim the track to the selected area only

def trim_items(track_index=0, start_time=0, end_time=1):
    # Create a new Reaper project object
    project = reapy.Project()

    # Check if the track index is valid
    if track_index < len(project.tracks):
        # Get the track by index
        track = project.tracks[track_index]

        # Get all items in the track
        items = track.items

        # Initialize a variable to track if any items were trimmed
        trimmed = False

        # Iterate over all items in the track
        for item in items:
            # Calculate the end time of the item
            item_end_time = item.position + item.length

            # Check if the item's start or end time falls within the specified range
            if item.position < end_time and item_end_time > start_time:
                # Calculate the new start and end positions for the item
                new_start = max(item.position, start_time)
                new_end = min(item_end_time, end_time)

                # Set the new item length to the trimmed range
                item.length = new_end - new_start

                # Set the new item position to the start time
                item.position = new_start

                # Set the trimmed flag to True
                trimmed = True

        # Check if any items were trimmed and print the appropriate message
        if trimmed:
            reapy.print("Items trimmed on track index: {}".format(track_index))
        else:
            reapy.print("No items found on track index: {}".format(track_index))
    else:
        reapy.print("Invalid track index.")

# Example usage

# Specify the track index, start time, and end time
track_index = 0
start_time = 5
end_time = 15


# Call the function to trim the specified part of the track
trim_items(track_index, start_time, end_time)


In [ ]:
#this function is to copy a part of a track

def copy_items(track_name='', start_time=0, end_time=1):
    project = reapy.Project()

    flag = False
    if track_name != '':
        for track in project.tracks:
            if track.name == track_name:
                flag = True
                items = track.items
                for item in items:
                    item_start_time = RPR.GetMediaItemInfo_Value(item.id, "D_POSITION")
                    item_end_time = item_start_time + RPR.GetMediaItemInfo_Value(item.id, "D_LENGTH")
                    if item.is_selected and (item_start_time >= start_time and item_end_time <= end_time):
                        item.copy()
                if flag:
                    reapy.print("Copying operation completed.")
                else:
                    reapy.print("No matching track found.")
                return
        if not flag:
            reapy.print("Track not found.")
    else:
        reapy.print("Specify the track name.")

# Example usage

# Get the track name
track_name = "My Track"

# Set the start and end time (in seconds)
start_time = 1
end_time = 2

# Call the function to copy the specified part of the track
copy_items(track_name, start_time, end_time)

In [ ]:
#this function is to cut a part of a track

def cut_items_within_range(track_index=0, start_time=0, end_time=1):
    # Get the current project
    project = reapy.Project()

    # Get the track by index
    track = project.tracks[track_index]

    # Get all items in the track
    items = track.items

    # Iterate over the items in reverse order to avoid index issues after cutting
    for item in reversed(items):
        item_start_time = item.position
        item_end_time = item.position + item.length

        # Check if the item falls within the specified time range
        if start_time <= item_start_time <= end_time and start_time <= item_end_time <= end_time:
            # Select the item
            item.select = True

    # Trigger the cut action in Reaper using the command ID
    RPR.Main_OnCommand(40006, 0)  # Cut selected items

    print("Items cut within the time range on track index: {}".format(track_index))

# Example usage

# Specify the track index, start time, and end time
track_index = 0
start_time = 1.0
end_time = 2.0

# Call the function to cut items within the specified time range
cut_items_within_range(track_index, start_time, end_time)


In [ ]:


# Paste
project = reapy.Project()

# Get the track where you want to paste the copied item
track = project.tracks[0]  # Replace [0] with the desired track index or use track name

# Check if there is an item in the clipboard
if project.has_copied_items():
    # Get the command ID for pasting
    paste_command_id = 41588

    # Execute the paste command on the track
    track.command(paste_command_id)
    print("Item pasted successfully.")
else:
    print("No item found in clipboard.")
